# Specific Memory Process Analysis
Now that we have identified a couple of processes to futher examine we will use the next set of plugins to analyze the process further and either determine if the process is malicious or generate futher indicators of compromise if we have already determined it is malicious

## handles
Handles are open kernel objects, common handles are the following:
* File
* Mutex
* Process 
* Registry key
* Thread

Before processes can interact with objects they need to open a handle by using Windows APIs. We have seen these API's before when reviewing unknown binaries. Common API calls to open handles are the following:
* CreateFile
* CreateMutex
* RegOpenKeyEx

When you call these API keys they retrun a file type called a HANDLE, which is an index to a process handle table. The handle count for the process is also incremented when the API call is made. API calls then can use the HANDLE value to pass to other API calls such as write, read and delete. When the function calls only refer to the HANDLE index it also helps with security through obscurity because it is just an index not the specific HANDLE name. 

Kernel modules and threads can also have open HANDLES, which are assigned to PID 4 process table. Some kernel API's to open handles are the following:

* NtCreateFile
* NtReadFile
* NtCreateMutex

To enumerate the handles table in volatilty we will use the handles plugin as shown below:


In [ ]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles

 Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0x823c8830      4        0x4   0x1f0fff Process          System(4)
0x823c7020      4        0x8        0x0 Thread           TID 12 PID 4
0xe1035468      4        0xc    0xf003f Key              MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\MEMORY MANAGEMENT\PREFETCHPARAMETERS
0xe1011490      4       0x10        0x0 Key              
0xe13af200      4       0x14    0x2001f Key              MACHINE\SYSTEM\SETUP
0xe13ae198      4       0x18    0x20019 Key              MACHINE\HARDWARE\DESCRIPTION\SYSTEM\MULTIFUNCTIONADAPTER
0xe101e438      4       0x1c    0x20019 Key              MACHINE\SYSTEM\WPA\MEDIACENTER
0xe13b0430      4       0x20    0x20019 Key              MACHINE\SYSTEM\WPA\KEY-4F3B2RFXKC9C637882MBM
0xe13b7430      4       0x24    0x20019 Key              MACHINE\SYSTEM\WPA\PNP
0xe13b8430      4       0x28    0x20019 Key        

That is too much to handle at first so lets just focus on our suspect PID, which is 796.

In [1]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles -p 796

Volatility Foundation Volatility Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0x821ace98    796        0xc   0x100020 File             \Device\HarddiskVolume1\WINDOWS\system32
0x82189028    796      0x578   0x12019f File             \Device\NamedPipe\ROUTER
0x822eff10    796      0x57c   0x1f0003 Event            
0x81f229d0    796      0x580   0x100003 Semaphore        
0x81f22a08    796      0x584   0x100003 Semaphore        
0x81f30838    796      0x588   0x100003 Semaphore        
0x822eae40    796      0x58c   0x100003 Semaphore        
0xe1c29c90    796      0x590    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
0xe1117fb8    796      0x594    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
0xe1c29c28    796      0x598    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
0xe1e9

You can also filter by handle type using the `-t` flag with the plugin. Below we just limit the handle output to registry keys and files.

In [2]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles -t Key,File

Volatility Foundation Volatility Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0xe1035468      4        0xc    0xf003f Key              MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\MEMORY MANAGEMENT\PREFETCHPARAMETERS
0xe1011490      4       0x10        0x0 Key              
0xe13af200      4       0x14    0x2001f Key              MACHINE\SYSTEM\SETUP
0xe13ae198      4       0x18    0x20019 Key              MACHINE\HARDWARE\DESCRIPTION\SYSTEM\MULTIFUNCTIONADAPTER
0xe101e438      4       0x1c    0x20019 Key              MACHINE\SYSTEM\WPA\MEDIACENTER
0xe13b0430      4       0x20    0x20019 Key              MACHINE\SYSTEM\WPA\KEY-4F3B2RFXKC9C637882MBM
0xe13b7430      4       0x24    0x20019 Key              MACHINE\SYSTEM\WPA\PNP
0xe13b8430      4       0x28    0x20019 Key              MACHINE\SYSTEM\WPA\SIGNINGHASH-V44KQMCFXKQCTQ
0xe13a6708      4       0x2c    0x2001f Key          

Some of the objects are not names and you can exclude these from analysis by using the `--silent` flag as shown below:

In [3]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 handles -p 796 --silent

Volatility Foundation Volatility Framework 2.6
Offset(V)     Pid     Handle     Access Type             Details
---------- ------ ---------- ---------- ---------------- -------
0x821ace98    796        0xc   0x100020 File             \Device\HarddiskVolume1\WINDOWS\system32
0x82189028    796      0x578   0x12019f File             \Device\NamedPipe\ROUTER
0xe1c29c90    796      0x590    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
0xe1117fb8    796      0x594    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
0xe1c29c28    796      0x598    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
0xe1e9f728    796      0x59c    0x20019 Key              MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\LINKAGE
0x81db29b0    796      0x5a8   0x1200a0 File             \Device\Ip
0x81db2a48    796      0x5ac   0x100003 File             \Device\Ip
0x81f0f7f0    796      0x5b0   0x1200a0 File       

## procdump

In [5]:
mkdir dump/



    Directory: C:\Users\jsmith\Documents\GitHub\IncidentResponse\processanalysis


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
d-----        4/10/2018   4:32 PM                dump                                                                  




In [7]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 procdump -D dump/ -p 796

Volatility Foundation Volatility Framework 2.6
Process(V) ImageBase  Name                 Result
---------- ---------- -------------------- ------
0x81dbdda0 0x00400000 iexplore.exe         OK: executable.796.exe


In [10]:
dir dump/



    Directory: C:\Users\jsmith\Documents\GitHub\IncidentResponse\processanalysis\dump


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
-a----        4/10/2018   4:32 PM         625664 executable.796.exe                                                    




## memdump

In [12]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 memdump -D dump/ -p 796

Volatility Foundation Volatility Framework 2.6
************************************************************************
Writing iexplore.exe [   796] to 796.dmp


In [13]:
dir dump/



    Directory: C:\Users\jsmith\Documents\GitHub\IncidentResponse\processanalysis\dump


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
-a----        4/10/2018   4:33 PM       98127872 796.dmp                                                               
-a----        4/10/2018   4:32 PM         625664 executable.796.exe                                                    




## strings
View the dump and process files in a strings utility such as FLOSS, viewing them in the notebook will freeze it from the amount of data.

## dlllist
Now that we have exporlered the process lets look at all the DLL's being loaded on the memory image. `dlllist` enumberates the doubly-linked list of `_LDR_DATA_TABLE_ENTRY` structures. DLLs added to the list when a process calls `LoadLibrary` funtion and are not unlisted `FreeLibrary` is called. The load count column lets you know if the DLL was statically or dynamically loaded.

In [14]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 dlllist

Volatility Foundation Volatility Framework 2.6
************************************************************************
System pid:      4
Unable to read PEB for task.
************************************************************************
smss.exe pid:    564
Command line : \SystemRoot\System32\smss.exe


Base             Size  LoadCount LoadTime                       Path
---------- ---------- ---------- ------------------------------ ----
0x48580000     0xf000     0xffff                                \SystemRoot\System32\smss.exe
0x7c900000    0xaf000     0xffff                                C:\WINDOWS\system32\ntdll.dll
************************************************************************
csrss.exe pid:    636
Command line : C:\WINDOWS\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,3072,512 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=winsrv:ConServerDllInitialization,2 ProfileControl=Off MaxRequ

You can specificy to look only specific processes with the `-p` flag as below with our supicious process

In [15]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 dlllist -p 796

Volatility Foundation Volatility Framework 2.6
************************************************************************
iexplore.exe pid:    796
Command line : "C:\Program Files\Internet Explorer\iexplore.exe"
Service Pack 3

Base             Size  LoadCount LoadTime                       Path
---------- ---------- ---------- ------------------------------ ----
0x00400000    0x9b000     0xffff                                C:\Program Files\Internet Explorer\iexplore.exe
0x7c900000    0xaf000     0xffff                                C:\WINDOWS\system32\ntdll.dll
0x7c800000    0xf6000     0xffff                                C:\WINDOWS\system32\kernel32.dll
0x77dd0000    0x9b000     0xffff                                C:\WINDOWS\system32\ADVAPI32.dll
0x77e70000    0x92000     0xffff                                C:\WINDOWS\system32\RPCRT4.dll
0x77fe0000    0x11000     0xffff                                C:\WINDOWS\system32\Secur32.dll
0x77f10000    0x49000     0xffff             

## dlldump
Lets dump all the DLL's associated with this process into our dump folder to futher examine them. You can do this with the `dlldump` command as shown below:

In [16]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 dlldump -D dump/ -p 796

Volatility Foundation Volatility Framework 2.6
Process(V) Name                 Module Base Module Name          Result
---------- -------------------- ----------- -------------------- ------
0x81dbdda0 iexplore.exe         0x000400000 iexplore.exe         OK: module.796.1fbdda0.400000.dll
0x81dbdda0 iexplore.exe         0x07c900000 ntdll.dll            OK: module.796.1fbdda0.7c900000.dll
0x81dbdda0 iexplore.exe         0x076b40000 WINMM.dll            OK: module.796.1fbdda0.76b40000.dll
0x81dbdda0 iexplore.exe         0x078130000 urlmon.dll           OK: module.796.1fbdda0.78130000.dll
0x81dbdda0 iexplore.exe         0x05ad70000 uxtheme.dll          OK: module.796.1fbdda0.5ad70000.dll
0x81dbdda0 iexplore.exe         0x076e80000 rtutils.dll          OK: module.796.1fbdda0.76e80000.dll
0x81dbdda0 iexplore.exe         0x010000000 irykmmww.d1l         OK: module.796.1fbdda0.10000000.dll
0x81dbdda0 iexplore.exe         0x0769c0000 USERENV.dll          OK: module.796.1fbdda0.769c0000.dll
0x8

Review the supicoius DLL, which is named `module.796.1fbdda0.10000000.dll` in your other tools sets in your VM.

## modules
`modules` lists all the loaded kernel drives on the system. This command enumerates the doubly-linked list of the  `LDR_DATA_TABLE_ENTRY` structures pointed to by PsLoadedModuleList which relives on finding the `KDBG` structure in memory similar to `pslist`

In [17]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 modules

Volatility Foundation Volatility Framework 2.6
Offset(V)  Name                 Base             Size File
---------- -------------------- ---------- ---------- ----
0x823fc3b0 ntoskrnl.exe         0x804d7000   0x1f8680 \WINDOWS\system32\ntkrnlpa.exe
0x823fc348 hal.dll              0x806d0000    0x20300 \WINDOWS\system32\hal.dll
0x823fc2e0 kdcom.dll            0xf8b9a000     0x2000 \WINDOWS\system32\KDCOM.DLL
0x823fc270 BOOTVID.dll          0xf8aaa000     0x3000 \WINDOWS\system32\BOOTVID.dll
0x823fc208 ACPI.sys             0xf856b000    0x2e000 ACPI.sys
0x823fc198 WMILIB.SYS           0xf8b9c000     0x2000 \WINDOWS\system32\DRIVERS\WMILIB.SYS
0x823fc130 pci.sys              0xf855a000    0x11000 pci.sys
0x823fc0c0 isapnp.sys           0xf869a000     0xa000 isapnp.sys
0x823fc050 compbatt.sys         0xf8aae000     0x3000 compbatt.sys
0x823ed008 BATTC.SYS            0xf8ab2000     0x4000 \WINDOWS\system32\DRIVERS\BATTC.SYS
0x823edf98 intelide.sys         0xf8b9e000     0x2000 intelide.sys

## modscan
As the naming convention follows `modscan` looks forensically for loaded modules. It will find loaded and unloaded modules. The list of the order of modules unlike modules is not the order they were loaded.

In [19]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 modscan

Volatility Foundation Volatility Framework 2.6
Offset(P)          Name                 Base             Size File
------------------ -------------------- ---------- ---------- ----
0x0000000001f1c7d0 irykmmww.sys         0xf836f000     0x4000 \??\C:\WINDOWS\system32\drivers\irykmmww.sys
0x0000000001fbd970 mrxdav.sys           0xf649b000    0x2d000 \SystemRoot\system32\DRIVERS\mrxdav.sys
0x0000000001fe34c8 ndiswan.sys          0xf82d2000    0x17000 \SystemRoot\system32\DRIVERS\ndiswan.sys
0x0000000001ff2568 watchdog.sys         0xf8a0a000     0x5000 \SystemRoot\System32\watchdog.sys
0x0000000001ff3338 dxg.sys              0xbf9c3000    0x12000 \SystemRoot\System32\drivers\dxg.sys
0x0000000001ff5630 dxgthk.sys           0xf8cba000     0x1000 \SystemRoot\System32\drivers\dxgthk.sys
0x0000000001ffb8c8 HTTP.sys             0xf6110000    0x41000 \SystemRoot\System32\Drivers\HTTP.sys
0x0000000002009670 mrxsmb.sys           0xf698e000    0x70000 \SystemRoot\system32\DRIVERS\mrxsmb.sys
0x000000

## unloadedmodules

In [20]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 unloadedmodules

Volatility Foundation Volatility Framework 2.6
Name                 StartAddress EndAddress Time
-------------------- ------------ ---------- ----
Sfloppy.SYS          0x00f8393000 0xf8396000 2009-04-16 16:10:00 
Cdaudio.SYS          0x00f89b2000 0xf89b7000 2009-04-16 16:10:00 
imapi.sys            0x00f88ba000 0xf88c5000 2009-04-16 16:10:00 


## moddump

In [22]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 moddump --regex=irykmmww.sys -D dump

Volatility Foundation Volatility Framework 2.6
Module Base Module Name          Result
----------- -------------------- ------
0x0f836f000 irykmmww.sys         OK: driver.f836f000.sys


Review the dumped modules in your tools on your VM, remember it is not named `irykmmww.sys` but `driver.f836f000.sys`